In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sklearn as sk
import seaborn as sns

In [ ]:
df= pd.read_csv('/content/drive/MyDrive/Praca_magisterska/LSTM/Natural_gas/NaturalGas_10Y.csv')
df

In [ ]:
df_wykres=df.copy()
df_wykres= df_wykres[['Date','Price']]
df_wykres.rename(columns={'Price': 'Close'}, inplace=True)
df_wykres['Date']=pd.to_datetime(df_wykres['Date'])
df_wykres = df_wykres.set_index('Date')
df_wykres

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(15,7))
sns.set_style("whitegrid")
plt.plot(df_wykres['Close'], label='Ceny zamknięcia')
plt.title("Ceny gazu ziemnego na przestrzeni lat", color = 'black', fontsize = 12)
plt.xlabel('Data', color = 'black', fontsize = 12)

plt.xticks(fontsize=10)
plt.ylabel('Cena surowca', color = 'black', fontsize = 15)
plt.yticks(fontsize=12)
plt.legend(fontsize=15);

In [ ]:
df = df.drop(['Vol.','Change %'], axis=1)
df

In [ ]:
df.isnull().values.any()

In [ ]:
df = df.rename(columns={'Price': 'Close'})
df

In [ ]:
df=df.loc[::-1].reset_index(drop=True)
df['Date']=pd.to_datetime(df['Date'])
df

In [ ]:
df_test = df.tail(30)
df_test.set_index('Date', inplace=True)
df_test = df_test.drop(columns=['Open','High','Low'])
df_test.to_csv('CrudeOil_test_df.csv', encoding='utf-8', index=True)
df_test

In [ ]:
df =df.iloc[:-30]
df

In [ ]:
Q3 = np.quantile(df['Close'], 0.75)
Q1 = np.quantile(df['Close'], 0.25)
IQR = Q3-Q1
lower_range = Q1 - 1.5 * IQR
upper_range = Q3 + 1.5 * IQR
outliers = [x for x in df['Close'] if ((x<lower_range) or (x>upper_range))]
result = len(outliers)

df_outliers = df[(df['Close'] < lower_range) | (df['Close'] > upper_range)]
df = df[(df['Close'] > lower_range) & (df['Close'] < upper_range)]

df=df.reset_index()
df = df.drop(['index'], axis=1)


In [ ]:
result

In [ ]:
df

In [ ]:
df_outliers.set_index('Date', inplace=True)

In [ ]:
plot = df_outliers['Close'].groupby(df_outliers.index.year).count().plot(kind="bar",title="Wartości odstające - węgiel kamienny", xlabel='Rok', ylabel='Ilość wartości odstających', figsize=(10,5))
plt.legend(["Ilość wartości odstających"])
plt.xticks(rotation=0)


In [ ]:
df['Month'] = df['Date'].dt.month
df

In [ ]:
df['Daily Variation']=((df['High']-df['Low'])/df['Open']).round(2)
df

In [ ]:
df['Change'] = (df['Close']-df['Close'].shift(1)).fillna(0)
df['Change%'] = (((df['Close']-df['Close'].shift(1))/df['Close'].shift(1))*100).fillna(0).round(2)
df

In [ ]:
df['7 Days SMA'] = df['Close'].rolling(7).mean().round(2)
df

In [ ]:
df['7 Days STD']=df['Close'].rolling(7).std().round(2)
df

In [ ]:
df['7 Days SMA +2STD'] = (df['7 Days SMA']+(2*df['7 Days STD'])).round(2)
df

In [ ]:
df['7 Days SMA -2STD'] = (df['7 Days SMA']-(2*df['7 Days STD'])).round(2)
df

In [ ]:
df['High-Close']=df['High']-df['Close']
df['Low-Open']=df['Low']-df['Open']
df

In [ ]:
df['Cumulative Change Sum'] = df['Change%'].cumsum().round(2)
df

In [ ]:
df['14 Days EMA'] = df['Close'].ewm(span=14, adjust=False).mean().round(2)
df

In [ ]:
df_RSI=df[['Close']].copy()
df_RSI['Change'] = (df_RSI['Close']-df_RSI['Close'].shift(1)).fillna(0)
df_RSI['Loss']=df_RSI['Change'].clip(upper=0)
df_RSI['Gain']=df_RSI['Change'].clip(lower=0)
df_RSI['Avg Loss']= df_RSI['Loss'].rolling(14).mean().round(2).abs()
df_RSI['Avg Gain']= df_RSI['Gain'].rolling(14).mean().round(2)
df_RSI['RS']=(df_RSI['Avg Gain']/df_RSI['Avg Loss']).round(2)
df_RSI['RSI']=(100-(100/(1+df_RSI['RS']))).round(2)
df_RSI
df['RSI']=df_RSI['RSI']


In [ ]:
df

In [ ]:
df_MACD=df[['Close']].copy()
df_MACD['12']=df['Close'].ewm(span=12, adjust=False).mean().round(2)
df_MACD['26']=df['Close'].ewm(span=26, adjust=False).mean().round(2)
df_MACD['MACD']=df_MACD['12']-df_MACD['26']
df['MACD']=df_MACD['MACD'].round(3)
df

In [ ]:
df['Stochastic Oscillator']=(((df['Close']-df['Close'].rolling(14).min())/(df['Close'].rolling(14).max()-df['Close'].rolling(14).min()))*100).round(2)
df

In [ ]:
df_ATR=df[['Close','High','Low']].copy()
df_ATR['H-L']=df_ATR['High']-df_ATR['Low']
df_ATR['H-C']=(df_ATR['High']-df_ATR['Close'].shift(1)).abs()
df_ATR['L-C']=(df_ATR['Low']-df_ATR['Close'].shift(1)).abs()
df_ATR['TR']=df_ATR[['H-L','H-C','L-C']].max(axis=1)
df_ATR



In [ ]:
TR_ar = df_ATR['TR'][13:].values
ATR_ar=[df_ATR['TR'][0:14].mean()]
ATR_ar
TR_ar


In [ ]:

for i in range(1,len(TR_ar)):
  ATR_ar.append((13*ATR_ar[i-1]+TR_ar[i])/14)

for j in range (len(df_ATR['TR'].values)-len(ATR_ar)):
  ATR_ar.insert(0,'NaN')

In [ ]:
df_ATR['ATR']=ATR_ar


In [ ]:
df_ATR

In [ ]:
df['ATR'] = df_ATR['ATR'].astype(float).round(3)

In [ ]:
df

In [ ]:
df.set_index('Date', inplace=True)

In [ ]:
scaler = sk.preprocessing.MinMaxScaler(feature_range=(0, 1))

In [ ]:
df_scaled=df.copy()

In [ ]:
df_scaled[df_scaled.columns] = scaler.fit_transform(df_scaled[df_scaled.columns])

In [ ]:
df_scaled

In [ ]:
df_scaled.corr(method='spearman', min_periods=100)

In [ ]:
df = df.drop(columns=['Stochastic Oscillator', 'Month', 'Daily Variation','Change', 'Change%','High-Close', 'Low-Open','RSI'])

In [ ]:
df=df.dropna()

In [ ]:
df

In [ ]:
df.to_csv('CrudeOil_df.csv', encoding='utf-8', index=True)